In [1]:
import sqlparse

from django.db import connection, reset_queries

In [2]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    

In [3]:
def get_published_posts(filtered_posts):
    queryset = filtered_posts
    # queryset = (
    #    queryset.prefetch_related("audios")
    #    .prefetch_related("images")
    #    .prefetch_related("videos")
    #    .prefetch_related("galleries")
    #    .select_related("owner")
    # )
    queryset = queryset.prefetch_related(
        "audios", "images", "videos", "galleries", "galleries__images", "owner",
        "images__renditions", "galleries__images__renditions",
    )
    return queryset


def get_all_renditions(posts):
    renditions = []
    for post in posts:
        for imag_type, image in post.get_all_images():
            for rendition in image.renditions.all():
                renditions.append(rendition)
    return renditions


In [4]:
blog = Blog.objects.get(slug="das_claas_und_nora_blog")
posts_queryset = get_published_posts(Post.objects.live().descendant_of(blog).order_by("-visible_date"))

In [5]:
%%time
reset_queries()
# posts = list(get_published_posts(posts_queryset))
renditions = list(Post.get_all_renditions_from_queryset(posts_queryset))
len(connection.queries)

CPU times: user 385 ms, sys: 28.8 ms, total: 414 ms
Wall time: 437 ms


9

In [6]:
len(renditions)

1810

In [7]:
Rendition.objects.count()

25835